Základní CNN s rozpoznáváním psaných číslic

In [9]:
import os
#!unzip -qq '/content/drive/MyDrive/Datafiles/it4n/cwe3categ.zip'
!wget -qq 'https://github.com/lukyfox/Datafiles/raw/master/digits/digits.zip'  
!unzip -qq '/content/digits.zip'
#!unzip -qq '/content/drive/MyDrive/Datafiles/it4n/it4n_train_reduced.zip'
#!unzip -qq '/content/drive/MyDrive/Datafiles/it4n/cwe3categ_augmented.zip'

replace digits/validation/1/img_41119.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [10]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, Callback
from keras import backend
from tensorflow.keras.applications import VGG16

from keras.metrics import Precision, Recall
from sklearn.metrics import classification_report

from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from collections import Counter

class AccuracyCallback(Callback):
    def __init__(self, test_data, classes):
        self.test_data = test_data
        self.classes = list(classes)

    def on_epoch_end(self, epoch, logs=None):
        x_data, y_data = self.test_data

        correct = 0
        incorrect = 0

        x_result = self.model.predict(x_data, verbose=0)

        class_correct = [0] * len(self.classes)
        class_incorrect = [0] * len(self.classes)

        for i in range(len(x_data)):
            x = x_data[i]
            y = y_data[i]

            res = x_result[i]

            actual_label = np.argmax(y)
            #print('actual_label =',actual_label)
            pred_label = np.argmax(res)
            #print('pred_label =', pred_label)

            if(pred_label == actual_label):
                class_correct[actual_label] += 1   
                correct += 1
            else:
                class_incorrect[actual_label] += 1
                incorrect += 1
        print('\tclass_correct =', class_correct)
        print('\tclass_incorrect =', class_incorrect)
        print("\tCorrect: %d" %(correct))
        print("\tIncorrect: %d" %(incorrect))

        for i in range(len(self.classes)):
            tot = float(class_correct[i] + class_incorrect[i])
            print(f'tot({tot}) = float(class_correct[i]({class_correct[i]}) + class_incorrect[i]({class_incorrect[i]}))')
            class_acc = -1
            if (tot > 0):
                class_acc = float(class_correct[i]) / tot
                print(f'class_acc({class_acc}) = float(class_correct[i])({class_correct[i]}) / tot({tot})')

            print("\tself.classes[i] = %s: class_acc = %.3f" %(self.classes[i],class_acc)) 

        acc = float(correct) / float(correct + incorrect)  
        print(f'acc({acc}) = float(correct)({correct}) / float(correct + incorrect)({correct+incorrect})')

        print("\tCurrent Network Accuracy: %.3f" %(acc))


In [11]:
# import relevant functions
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, Adam

# rotate, and shift data
#image_gen = ImageDataGenerator(rotation_range=20,
#                              width_shift_range=0.1,
#                              height_shift_range=0.1,
#                              shear_range=0.1,
#                              zoom_range=0.1,
#                              horizontal_flip=True,
#                              fill_mode='nearest')

image_gen = ImageDataGenerator(rescale=1.0/255)
# build the model

#image_shape = (330, 330, 3)
image_shape = (32, 32, 3)


from keras.applications.vgg16 import VGG16
conv_base = VGG16(weights='imagenet', include_top=False, input_shape=image_shape)

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=image_shape, activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

learn_rate=0.001
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# set up early stopping - overfitting solution + speed up processing
early_stopping = EarlyStopping(monitor='val_loss', patience=4)

batch_size=16

# build train and test generators
#train_path = '/content/_splitted/train'
train_path = '/content/digits/train'
#train_path = '/content/cwe3categ_augmented/train'
train_image_gen = image_gen.flow_from_directory(train_path,
                                               target_size = image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical')

# don't shuffle test data
#test_path = '/content/_splitted/validation'
test_path = '/content/digits/validation'
#test_path = '/content/cwe3categ_augmented/validation'

test_image_gen = image_gen.flow_from_directory(test_path,
                                               target_size = image_shape[:2],
                                               color_mode='rgb',
                                               batch_size=batch_size,
                                               class_mode='categorical',
                                               shuffle=False)

accuracy_callback = AccuracyCallback(test_image_gen, [0,1,2])

# fit on the generators
results = model.fit(train_image_gen, epochs=20,
                             validation_data=test_image_gen,
                             #callbacks=[early_stopping, accuracy_callback]
                    )
results.history

58900480/58889256 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Found 9880 images belonging to 3 classes.
Found 3113 images belonging to 3 classes.
Epoch 1/20
618/618 [==============================] - 25s 13ms/step - loss: 0.0770 - accuracy: 0.9720 - val_loss: 0.0325 - val_accuracy: 0.9900
Epoch 2/20
618/618 [==============================] - 7s 12ms/step - loss: 0.0216 - accuracy: 0.9928 - val_loss: 0.0177 - val_accuracy: 0.9945
Epoch 3/20
618/618 [==============================] - 7s 12ms/step - loss: 0.0135 - accuracy: 0.9960 - val_loss: 0.0110 - val_accuracy: 0.9981
Epoch 4/20
618/618 [==============================] - 7s 12ms/step - loss: 0.0117 - accuracy: 0.9963 - val_loss: 0.0053 - val_accuracy: 0.9981
Epoch 5/20
618/618 [==============================] - 7s 12ms/step - loss: 0.0104 - accuracy: 0.9965 - val_loss: 0.0093 - val_accuracy: 0.9965
Epoch 6/20
618/618 [==============================] - 7s 12ms/step - loss: 0.0037 - accuracy: 0.9989 - val_loss: 0.0047 - val_accuracy: 0.9990
Epoch 7/20
618/618 [==============================] - 7s 

{'accuracy': [0.9719635844230652,
  0.992813766002655,
  0.9959514141082764,
  0.9962550401687622,
  0.9964575171470642,
  0.9988866448402405,
  0.9979757070541382,
  0.9976720809936523,
  0.9987854361534119,
  0.9997975826263428,
  0.9963562488555908,
  0.9985830187797546,
  0.9991902709007263,
  0.9987854361534119,
  0.9993926882743835,
  1.0,
  0.9987854361534119,
  0.9994939565658569,
  0.9990890622138977,
  0.9997975826263428],
 'loss': [0.0769883543252945,
  0.021585850045084953,
  0.01351700909435749,
  0.011718499474227428,
  0.010437392629683018,
  0.003656608983874321,
  0.005942370742559433,
  0.007743736729025841,
  0.0037473735865205526,
  0.0013580884551629424,
  0.012825550511479378,
  0.0034016799181699753,
  0.002439531497657299,
  0.0033627671655267477,
  0.0028568259440362453,
  5.993447848595679e-05,
  0.005846542306244373,
  0.003347464371472597,
  0.0043944926001131535,
  0.0006919779116287827],
 'val_accuracy': [0.9900417327880859,
  0.9945390224456787,
  0.99807